In [2]:
import pygame
import sys
import time

# Pygame 초기화
pygame.init()
width, height = 1200, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("슈의 라면가게 - 마우스 버전")

# 폰트 & 색상
font = pygame.font.SysFont("malgungothic", 30)
title_font = pygame.font.SysFont("malgungothic", 50, True)
WHITE = (255,255,255)
BLACK = (0,0,0)
GRAY = (200,200,200)
RED = (255,0,0)
GREEN = (0,255,0)
BLUE = (0,0,255)
PURPLE = (128,0,128)

# 냄비 클래스
class Pot:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 120, 80)
        self.state = 0  # 0: 빈 냄비, 1: 물, 2: 면, 3: 스프, 4: 파, 5: 계란
        self.cook_start = 0
        self.order = []
        self.dragging = False
        self.done = False

    def add_ingredient(self, ingredient):
        if self.state == 0 and ingredient == "물":
            self.state = 1
            self.cook_start = time.time()
        elif self.state >=1:
            self.order.append(ingredient)
            self.state +=1

    def get_price(self):
        # 순서 체크
        correct_order = ["면", "스프", "파", "계란"]
        if self.order != correct_order:
            return 2000
        # 시간 체크
        cook_time = time.time() - self.cook_start
        if cook_time >= 10:
            return 0
        elif cook_time >=4:
            return 3000
        elif cook_time >=2:
            return 1000
        else:
            return 0

    def draw(self, screen):
        color = GRAY if self.state ==0 else (150,150,255)
        pygame.draw.rect(screen, color, self.rect, 0, 10)
        pygame.draw.rect(screen, BLACK, self.rect, 2, 10)
        # 상태 텍스트
        state_text = ["빈 냄비", "물", "면", "스프", "파", "계란"][self.state] if self.state <=5 else "탔음!"
        text = font.render(state_text, True, BLACK)
        screen.blit(text, (self.rect.x+10, self.rect.y-25))

# UI 버튼 클래스
class Button:
    def __init__(self, x, y, text, action=None):
        self.rect = pygame.Rect(x, y, 150, 50)
        self.text = text
        self.action = action
        self.color = (200,200,200)

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect, 0, 5)
        text = font.render(self.text, True, BLACK)
        screen.blit(text, (self.rect.x+20, self.rect.y+15))

# 게임 관리 클래스
class Game:
    def __init__(self):
        self.pots = [Pot(200,400), Pot(500,400)]
        self.buttons = [
            Button(800, 100, "물 붓기", "물"),
            Button(800, 170, "면 넣기", "면"),
            Button(800, 240, "스프 넣기", "스프"),
            Button(800, 310, "파 넣기", "파"),
            Button(800, 380, "계란 넣기", "계란"),
            Button(800, 500, "완성!", "완성")
        ]
        self.score = 0
        self.stage = 1
        self.time_limit = 90
        self.start_time = time.time()
        self.selected_pot = None

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            
            # 마우스 이벤트 처리
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                # 버튼 클릭 체크
                for btn in self.buttons:
                    if btn.rect.collidepoint(pos):
                        if btn.action == "완성":
                            for pot in self.pots:
                                if pot.rect.collidepoint(pos) and pot.state >0:
                                    self.score += pot.get_price()
                                    pot.__init__(pot.rect.x, pot.rect.y)
                        else:
                            for pot in self.pots:
                                if pot.rect.collidepoint(pos):
                                    pot.add_ingredient(btn.action)
                # 냄비 드래그 시작
                for pot in self.pots:
                    if pot.rect.collidepoint(pos):
                        self.selected_pot = pot
                        pot.dragging = True
            
            elif event.type == pygame.MOUSEBUTTONUP:
                if self.selected_pot:
                    self.selected_pot.dragging = False
                    self.selected_pot = None
            
            elif event.type == pygame.MOUSEMOTION:
                if self.selected_pot and self.selected_pot.dragging:
                    self.selected_pot.rect.x += event.rel[0]
                    self.selected_pot.rect.y += event.rel[1]

    def draw(self, screen):
        screen.fill(WHITE)
        # 냄비 그리기
        for pot in self.pots:
            pot.draw(screen)
        # UI 버튼 그리기
        for btn in self.buttons:
            btn.draw(screen)
        # 정보 패널
        time_left = self.time_limit - (time.time()-self.start_time)
        pygame.draw.rect(screen, GRAY, (50,50,300,100))
        info_text = [
            f"스테이지: {self.stage}",
            f"목표: {[10000,12000,15000][self.stage-1]}원",
            f"시간: {int(time_left)}초",
            f"점수: {self.score}원"
        ]
        for i, text in enumerate(info_text):
            txt = font.render(text, True, BLACK)
            screen.blit(txt, (70, 60+i*25))
        # 보라색 쟁반
        pygame.draw.rect(screen, PURPLE, (1000,400,150,100))
        pygame.display.update()

    def run(self):
        while True:
            self.handle_events()
            self.draw(screen)
            # 시간 초과 체크
            if time.time() - self.start_time > self.time_limit:
                if self.score >= [10000,12000,15000][self.stage-1]:
                    self.stage +=1
                    self.start_time = time.time()
                    self.time_limit = [90,60,45][self.stage-1]
                else:
                    print("게임 오버!")
                    return

# 게임 실행
game = Game()
game.run()


SystemExit: 